In [12]:
import tensorflow as tf
from src.robot_reboot.dataset_factory import RobotRebootDataSetFactory
tf.__version__

'2.1.2'

In [2]:
cnn = tf.keras.models.load_model('model/model_0')

In [3]:
ds_factory = RobotRebootDataSetFactory(31, cnn, max_depth=20, seed=26, playouts=50)
v, p, s = ds_factory.create(locate_robot_close_goal=True, max_movements=3)

In [30]:
rows, cols, layers = 31, 31, 9
state = s.reshape(rows * cols * layers,).astype('int64') 
state

array([0, 0, 0, ..., 0, 0, 0])

### Serialize

In [33]:
sample = tf.train.Example(
    features = tf.train.Features(
        feature = {
            'v': tf.train.Feature(float_list=tf.train.FloatList(value=[v])),
            'p': tf.train.Feature(float_list=tf.train.FloatList(value = p)),
            's': tf.train.Feature(int64_list=tf.train.Int64List(value = state))
        }
    )
)

In [35]:
with tf.io.TFRecordWriter('tf_record_data') as f:
    f.write(sample.SerializeToString())

### Deserialize

In [47]:
feature_desc = {
    'v': tf.io.FixedLenFeature([], tf.float32),
    'p': tf.io.VarLenFeature(tf.float32),
    's': tf.io.VarLenFeature(tf.int64)
}

In [49]:
for serialized in tf.data.TFRecordDataset(['tf_record_data']):
    parsed = tf.io.parse_single_example(serialized, feature_desc)
    print(parsed)

{'p': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7f28360d69e8>, 's': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7f283606ada0>, 'v': <tf.Tensor: shape=(), dtype=float32, numpy=0.0>}


### Train NN

In [63]:
def read_tfrecord(example):
    feature_desc = {
        'v': tf.io.FixedLenFeature([], tf.float32),
        'p': tf.io.VarLenFeature(tf.float32),
        's': tf.io.VarLenFeature(tf.int64)
    }
    sample = tf.io.parse_single_example(example, feature_desc)
    y = 
    return sample['s'], {'v':sample['v'], 'p':sample['p']}            

SyntaxError: invalid character in identifier (<ipython-input-63-66e67e1d9a02>, line 9)

In [58]:
ds = tf.data.TFRecordDataset(['tf_record_data'])
ds = ds.map(read_tfrecord)

SparseTensor(indices=Tensor("ParseSingleExample/ParseExample/ParseExampleV2:1", shape=(None, 1), dtype=int64), values=Tensor("ParseSingleExample/ParseExample/ParseExampleV2:3", shape=(None,), dtype=int64), dense_shape=Tensor("ParseSingleExample/ParseExample/ParseExampleV2:5", shape=(1,), dtype=int64))


In [56]:
cnn.fit(ds, epochs = 1)

ValueError: in converted code:

    /home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/tensorflow_core/python/keras/engine/training_v2.py:677 map_fn
        batch_size=None)
    /home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/tensorflow_core/python/keras/engine/training.py:2428 _standardize_tensors
        exception_prefix='input')
    /home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/tensorflow_core/python/keras/engine/training_utils.py:573 standardize_input_data
        'with shape ' + str(data_shape))

    ValueError: Error when checking input: expected input to have 4 dimensions, but got array with shape ()


In [57]:
read_tfrecord(ds)

ValueError: Attempt to convert a value (<MapDataset shapes: ((), (None,), (None,)), types: (tf.float32, tf.float32, tf.int64)>) with an unsupported type (<class 'tensorflow.python.data.ops.dataset_ops.MapDataset'>) to a Tensor.

In [60]:
cnn.fit(x=[state], y = {'v': [v], 'p': [p]})

ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {"<class 'numpy.ndarray'>"}), (<class 'dict'> containing {"<class 'str'>"} keys and {'(<class \'list\'> containing values of types {"<class \'int\'>"})', '(<class \'list\'> containing values of types {"<class \'numpy.ndarray\'>"})'} values)

In [61]:
np.array(v)

NameError: name 'np' is not defined